In [ ]:
import os, sys
os.system('source /cms/ldap_home/yeonjoon/working_dir/VcbMVAStudy/setup.sh')

: 

In [2]:
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import sys
sys.path.append('/cms/ldap_home/yeonjoon/working_dir/VcbMVAStudy')
from root_data_loader import load_data, classWtoSampleW
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

/cms/ldap_home/yeonjoon/miniconda3/envs/ML-torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Welcome to JupyROOT 6.26/06
True
Tesla P100-PCIE-16GB
1
cuda


In [3]:
varlist = ['bvsc_w_u','bvsc_w_d','cvsl_w_u','cvsl_w_d','cvsb_w_u','cvsb_w_d','n_bjets','n_cjets','weight']
varlist.extend(['n_jets',
                'pt_had_t_b','pt_w_u','pt_w_d','pt_lep_t_b',
                'eta_had_t_b','eta_w_u','eta_w_d','eta_lep_t_b',
                'bvsc_lep_t_b','bvsc_had_t_b'])

In [4]:
path_sample = 'root://cluster142.knu.ac.kr//store/user/yeonjoon'
#filename = 'Vcb_Mu_TTLJ_WtoCB_powheg_25.root'
filename = 'Vcb_2018_Mu_Reco_Tree.root' 
#,'Reco_41','Reco_23','Reco_21'
data =  load_data(file_path=os.path.join(path_sample,filename),varlist=varlist,test_ratio=0,val_ratio=0.2,sigTree=['Reco_45'],bkgTree=['Reco_43','Reco_41','Reco_23','Reco_21'])
 

root://cluster142.knu.ac.kr//store/user/yeonjoon/Vcb_2018_Mu_Reco_Tree.root
          bvsc_w_u  bvsc_w_d  cvsl_w_u  cvsl_w_d  cvsb_w_u  cvsb_w_d  n_bjets  \
0         0.005346  0.004351  0.037492  0.032832  0.874620  0.882540        2   
1         0.009966  0.214181  0.050486  0.540041  0.833751  0.664585        2   
2         0.004412  0.046763  0.033420  0.105244  0.882933  0.682070        2   
3         0.021966  0.349963  0.088058  0.620084  0.796780  0.535266        2   
4         0.008350  0.466603  0.079766  0.336992  0.904520  0.278099        2   
...            ...       ...       ...       ...       ...       ...      ...   
33343479  0.003910  0.008875  0.030518  0.047035  0.886033  0.840065        2   
33343480  0.017368  0.003610  0.084502  0.033819  0.827017  0.903243        2   
33343481  0.012735  0.035011  0.071595  0.126640  0.847334  0.777307        2   
33343482  0.012129  0.010957  0.068448  0.051854  0.847907  0.823959        2   
33343483  0.005697  0.012665  0.0

In [5]:
clf = TabNetClassifier(
    n_d=16,
    n_a=16,
    verbose=1,
    cat_idxs=data['cat_idxs'],
    cat_dims=data['cat_dims'],
    cat_emb_dim=1
    )  
from pytorch_tabnet.augmentations import ClassificationSMOTE
aug = ClassificationSMOTE(p=0.5)

/cms/ldap_home/yeonjoon/miniconda3/envs/ML-torch/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [6]:
clf.fit(
    X_train=data['train_features'],y_train=data['train_y'],
    eval_set=[(data['val_features'], data['val_y'])],
    eval_metric=['auc'],
    max_epochs=10,
    num_workers=4,
    #weights=data['class_weight']
    weights=0,
    batch_size=8192*4,
    virtual_batch_size=2048*8,
    augmentations=aug
    #callbacks=[pytorch_tabnet.callbacks.History(clf,verbose=1)]
  )

100%|██████████| 814/814 [01:14<00:00, 10.96it/s]


epoch 0  | loss: 0.03101 | val_0_auc: 0.797   |  0:01:34s


100%|██████████| 814/814 [01:15<00:00, 10.74it/s]


epoch 1  | loss: 0.02722 | val_0_auc: 0.80814 |  0:03:10s


100%|██████████| 814/814 [01:16<00:00, 10.59it/s]


...

100%|██████████| 814/814 [01:12<00:00, 11.29it/s]


epoch 8  | loss: 0.02694 | val_0_auc: 0.80589 |  0:13:41s


100%|██████████| 814/814 [01:16<00:00, 10.67it/s]


epoch 9  | loss: 0.02713 | val_0_auc: 0.80783 |  0:15:18s
Stop training because you reached max_epochs = 10 with best_epoch = 4 and best_val_0_auc = 0.81548
eval
eval end


/cms/ldap_home/yeonjoon/miniconda3/envs/ML-torch/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [7]:
from matplotlib import pyplot as plt

In [8]:
modelist = ['45','43','41','23','21']
# varlist.extend(['n_jets',
#                 'pt_had_t_b','pt_w_u','pt_w_d','pt_lep_t_b',
#                 'eta_had_t_b','eta_w_u','eta_w_d','eta_lep_t_b',
#                 'bvsc_lep_t_b','bvsc_had_t_b',
#                 'm_w_u','m_w_d'])
model = TabNetClassifier()
model.load_model('/cms/ldap_home/yeonjoon/working_dir/VcbMVAStudy/TabNet_template/model.pt.zip')
for mode in modelist:
    data =  load_data(file_path='root://cluster142.knu.ac.kr//store/user/yeonjoon/Vcb_2018_Mu_Reco_Tree.root',varlist=varlist,test_ratio=0,val_ratio=0,sigTree=[f'Reco_{mode}'],bkgTree=[])
    arr = data['train_features']
    plt.hist(model.predict_proba(arr)[:,1],bins=40)
    plt.savefig(f'/cms/ldap_home/yeonjoon/working_dir/VcbMVAStudy/TabNet_template/result_{mode}_temp.png')
    plt.clf()

    del arr

root://cluster142.knu.ac.kr//store/user/yeonjoon/Vcb_2018_Mu_Reco_Tree.root
Full dataset, For validation
        bvsc_w_u  bvsc_w_d  cvsl_w_u  cvsl_w_d  cvsb_w_u  cvsb_w_d  n_bjets  \
0       0.005346  0.004351  0.037492  0.032832  0.874620  0.882540        2   
1       0.009966  0.214181  0.050486  0.540041  0.833751  0.664585        2   
2       0.004412  0.046763  0.033420  0.105244  0.882933  0.682070        2   
3       0.021966  0.349963  0.088058  0.620084  0.796780  0.535266        2   
4       0.008350  0.466603  0.079766  0.336992  0.904520  0.278099        2   
...          ...       ...       ...       ...       ...       ...      ...   
164275  0.117628  0.514092  0.999177  0.562011  0.882286  0.346918        3   
164276  0.009321  0.568587  0.049498  0.993976  0.840274  0.429932        2   
164277  0.030051  0.975839  0.079449  0.985171  0.719441  0.023811        3   
164278  0.025305  0.793889  0.077474  0.997660  0.749007  0.205728        2   
164279  0.035696  0.564575

<Figure size 640x480 with 0 Axes>